In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
import json
import csv
import random
import requests

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
from nltk.stem import WordNetLemmatizer
wnl = WordNetLemmatizer()
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
import pickle

In [ ]:
url = "https://raw.githubusercontent.com/Setiawan2800/ML-Capstone/main/ML-Capstone/medical-related.json"

response = requests.get(url)

if response.status_code == 200:
    data = json.loads(response.text)
    print(data)
else:
    print(f"Error: {response.status_code}")


{'intents': [{'tag': 'Cuts', 'patterns': ['What to do if Cuts?', 'How to cure Cuts?', 'Which medicine to apply for Cuts?', 'what to apply on cuts?', 'Cuts'], 'responses': ['Wash the cut properly to prevent infection and stop the bleeding by applying pressure for 1-2minutes until bleeding stops. Apply Petroleum Jelly to make sure that the wound is moist for quick healing. Finally cover the cut with a sterile bandage. Pain relievers such as acetaminophen can be applied.'], 'context_set': ''}, {'tag': 'Abrasions', 'patterns': ['how do you treat abrasions?', 'Do Abrasions cause scars?', 'Abrasions', 'what to do if abrasions?', 'Which medicine to apply for abrasions?', 'How to cure abrasions?'], 'responses': ['Begin with washed hands.Gently clean the area with cool to lukewarm water and mild soap. Remove dirt or other particles from the wound using sterilized tweezers.For a mild scrape that’s not bleeding, leave the wound uncovered.If the wound is bleeding, use a clean cloth or bandage, and

In [ ]:
print(data)

{'intents': [{'tag': 'Cuts', 'patterns': ['What to do if Cuts?', 'How to cure Cuts?', 'Which medicine to apply for Cuts?', 'what to apply on cuts?', 'Cuts'], 'responses': ['Wash the cut properly to prevent infection and stop the bleeding by applying pressure for 1-2minutes until bleeding stops. Apply Petroleum Jelly to make sure that the wound is moist for quick healing. Finally cover the cut with a sterile bandage. Pain relievers such as acetaminophen can be applied.'], 'context_set': ''}, {'tag': 'Abrasions', 'patterns': ['how do you treat abrasions?', 'Do Abrasions cause scars?', 'Abrasions', 'what to do if abrasions?', 'Which medicine to apply for abrasions?', 'How to cure abrasions?'], 'responses': ['Begin with washed hands.Gently clean the area with cool to lukewarm water and mild soap. Remove dirt or other particles from the wound using sterilized tweezers.For a mild scrape that’s not bleeding, leave the wound uncovered.If the wound is bleeding, use a clean cloth or bandage, and

In [ ]:
tags = []
patterns = []
responses = {}
for intent in data['intents']:
    responses[intent['tag']] = intent['responses']
    for lines in intent['patterns']:
        patterns.append(lines)
        tags.append(intent['tag'])

data = pd.DataFrame({'patterns':patterns,'tags':tags})

In [ ]:
data

,patterns,tags
0,What to do if Cuts?,Cuts
1,How to cure Cuts?,Cuts
2,Which medicine to apply for Cuts?,Cuts
3,what to apply on cuts?,Cuts
4,Cuts,Cuts
...,...,...
183,How to help a drowning person in CPR?,CPR
184,How do you treat a Fracture?,Fracture
185,what to do if i get a Fracture?,Fracture
186,Which medicine to take if I have a Fracture?,Fracture


In [ ]:
import string
data['patterns'] = data['patterns'].apply(lambda word:[ltrs.lower() for ltrs in word if ltrs not in string.punctuation])
data['patterns'] = data['patterns'].apply(lambda word:''.join(word))
data

,patterns,tags
0,what to do if cuts,Cuts
1,how to cure cuts,Cuts
2,which medicine to apply for cuts,Cuts
3,what to apply on cuts,Cuts
4,cuts,Cuts
...,...,...
183,how to help a drowning person in cpr,CPR
184,how do you treat a fracture,Fracture
185,what to do if i get a fracture,Fracture
186,which medicine to take if i have a fracture,Fracture


In [ ]:
class LabelEncoder:
    def __init__(self):
        self.label_dict = {}
        self.inverse_label_dict = {}

    def fit(self, y):
        unique_labels = set(y)
        self.label_dict = {label: index for index, label in enumerate(unique_labels)}
        self.inverse_label_dict = {index: label for label, index in self.label_dict.items()}

    def transform(self, y):
        return [self.label_dict[label] for label in y]

    def fit_transform(self, y):
        self.fit(y)
        return self.transform(y)

    def inverse_transform(self, y):
        return [self.inverse_label_dict[index] for index in y]

    @property
    def classes_(self):
        return list(self.label_dict.keys())

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=2000)
tokenizer.fit_on_texts(data['patterns'])
train = tokenizer.texts_to_sequences(data['patterns'])

x_train =  pad_sequences(train)

le = LabelEncoder()
y_train = le.fit_transform(data['tags'])

In [ ]:
input_shape = x_train.shape[1]
print(input_shape)

11


In [ ]:
vocabulary = len(tokenizer.word_index)
print("number of unique words : ",vocabulary)
output_length = len(le.classes_)
print("output length : ",output_length)

number of unique words :  131
output length :  44


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,Embedding, LSTM, Dense, GlobalMaxPooling1D, Flatten

i = Input(shape=(input_shape,))
x = Embedding(vocabulary+1,10)(i)
x = LSTM(10,return_sequences=True)(x)
x = Flatten()(x)
x = Dense(output_length,activation="softmax")(x)
model = Model(i,x)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

y_train = np.array(y_train)
x_train = x_train.astype(np.int32)
y_train = y_train.astype(np.int32)

train = model.fit(x_train,y_train,epochs=200)

Epoch 1/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.0188 - loss: 3.7854
Epoch 2/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0233 - loss: 3.7822 
Epoch 3/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0335 - loss: 3.7806     
Epoch 4/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0694 - loss: 3.7773 
Epoch 5/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0623 - loss: 3.7754  
Epoch 6/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.0889 - loss: 3.7724 
Epoch 7/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.1123 - loss: 3.7699 
Epoch 8/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0890 - loss: 3.7682 
Epoch 9/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0991 - loss: 3.7592 
Epoch 10/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.0950 - loss: 3.7556 
Epoch 11/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0859 - loss: 3.7520 
Epoch 12/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0980 -

In [ ]:
import random

while True:
  texts_p = []
  prediction_input = input('You: ')
  prediction_input = [letters.lower() for letters in prediction_input if letters not in string.punctuation]
  prediction_input = ''.join(prediction_input)
  texts_p.append(prediction_input)
  prediction_input = tokenizer.texts_to_sequences(texts_p)
  prediction_input = np.array(prediction_input).reshape(-1)
  prediction_input = pad_sequences([prediction_input],input_shape)
  output = model.predict(prediction_input)
  output = output.argmax()
  response_tag = le.inverse_transform([output])[0]
  print("Bot: ",random.choice(responses[response_tag]))
  if response_tag == "goodbye":
    break

You: i got cuts
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/step
Bot:  Wash the cut properly to prevent infection and stop the bleeding by applying pressure for 1-2minutes until bleeding stops. Apply Petroleum Jelly to make sure that the wound is moist for quick healing. Finally cover the cut with a sterile bandage. Pain relievers such as acetaminophen can be applied.
You: ihave fever
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Bot:  To treat a fever at home: 1)Drink plenty of fluids to stay hydrated. 2)Dress in lightweight clothing. 3)Use a light blanket if you feel chilled, until the chills end. 4)Take acetaminophen (Tylenol, others) or ibuprofen (Advil, Motrin IB, others). 5) Get medical help if the fever lasts more than five days in a row.
You: i got stings
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Bot:  Remove any stingers immediately. Some experts recommend scraping out the stinger with a credit card. Applying ice to the site may provide some mild relief. Apply ice for 20 minutes once every hour as 